<a href="https://colab.research.google.com/github/KhanhPham2411/DeepLearning_BaNgoc_Protonx/blob/master/Hackathon/03/Hack_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!wget --no-check-certificate \
    https://storage.googleapis.com/protonx-cloud-storage/data.txt
data = open('data.txt').read()

--2020-08-14 14:52:30--  https://storage.googleapis.com/protonx-cloud-storage/data.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 74.125.23.128, 74.125.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘data.txt.3’

data.txt.3          100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-08-14 14:52:30 (124 MB/s) - ‘data.txt.3’ saved [93578/93578]



In [17]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [83]:
corpus = data.lower().split("\n")

In [ ]:
corpus

## Yêu cầu: Sinh từ tiếp khi gõ bất cứ một câu nào có chiều dài bất kỳ.

In [ ]:
# Dự đoán 10 từ tiếp theo
# despite of wrinkles -> this thy golden time to heart's sight ' must '

### 1. Xử lý dữ liệu. Chia features, label

In [84]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [85]:
total_words

3211

In [86]:
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

### 2. Xây dựng model
#### Yêu cầu độ chính xác: 80%

In [88]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM, Bidirectional, Dropout

In [89]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(150))
model.add(Dense(512, activation='relu'))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.00025)

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 10, 100)           321100    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 10, 300)           301200    
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 300)           0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 150)               270600    
_________________________________________________________________
dense_14 (Dense)             (None, 512)               77312     
_________________________________________________________________
dense_15 (Dense)             (None, 3211)              1647243   
Total params: 2,617,455
Trainable params: 2,617,455
Non-trainable params: 0
___________________________________________

In [93]:
adam = Adam(lr=0.00025)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])


In [91]:
DESIRED_ACCURACY = 0.8

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>DESIRED_ACCURACY):
            print("\nReached 80% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [94]:
history = model.fit(xs, ys, epochs=1000, shuffle=True, verbose=1, callbacks=[callbacks])

Epoch 1/1000
483/484 [============================>.] - ETA: 0s - loss: 0.6922 - acc: 0.8078
Reached 80% accuracy so cancelling training!
484/484 [==============================] - 12s 24ms/step - loss: 0.6923 - acc: 0.8078


### 3. Dự đoán 10 từ tiếp theo

In [95]:
test_seq = 'despite of wrinkles'
# sinh ra 10 từ tiếp theo sau test_seq
# despite of wrinkles this thy golden time to heart's sight ' must '
next_words = 10
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([test_seq])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	test_seq += " " + output_word
print(test_seq)

despite of wrinkles this thy golden time spread fired blind lie exceeds '
